In [3]:
# Read the input file and format the content
input_file = "/data/QAAPI/requirements2_txtfiles.txt"  # Replace with your file name
output_file = "requirements_txtfiles.txt"


with open(input_file, "r") as file:
    lines = file.readlines()

# Extract and reformat lines
formatted_lines = []
for line in lines:
    if "==" in line:  # Check for package entries
        package, version = line.strip().split("==")
        formatted_lines.append(f"{package}=={version}\n")

# Write the reformatted lines to the output file
with open(output_file, "w") as file:
    file.writelines(formatted_lines)

print(f"Reformatted content has been saved to {output_file}")


Reformatted content has been saved to requirements_txtfiles.txt


In [4]:
import os
import subprocess

def extract_from_doc(file_path):
    try:
        # Determine the file extension
        file_ext = os.path.splitext(file_path)[1].lower()

        if file_ext == '.doc':
            # Convert .doc file using unoconv
            command = ['unoconv', '--format', 'txt', '--stdout', file_path]
        elif file_ext == '.docx':
            # Convert .docx file using unoconv
            command = ['unoconv', '--format', 'txt', '--stdout', file_path]
        else:
            raise ValueError(f"Unsupported file extension: {file_ext}. Only .doc and .docx files are supported.")

        # Run the unoconv command as a subprocess and capture the output
        result = subprocess.run(command, capture_output=True, text=True, check=True)

        # Get the text content from the stdout of unoconv
        text_content = result.stdout

        print(f"Successfully converted {file_path} to text using unoconv.")

        return text_content

    except subprocess.CalledProcessError as e:
        print(f"Failed to convert {file_path} with unoconv. Return code: {e.returncode}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")


In [5]:
extract_from_doc("data/tendergpt/livetender/73715367/doc173715367716582683.doc")

Failed to convert data/tendergpt/livetender/73715367/doc173715367716582683.doc with unoconv. Return code: 6


In [6]:
import subprocess

def extract_from_doc(file_path):
    try:
        command = ['antiword', file_path]
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        text_content = result.stdout
        print(f"Successfully converted {file_path} to text using antiword.")
        return text_content
    except subprocess.CalledProcessError as e:
        print(f"Failed to convert {file_path} with antiword. Return code: {e.returncode}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

text_content = extract_from_doc("data/tendergpt/livetender/73715367/doc173715367716582683.doc")
print(text_content)


An error occurred: [Errno 2] No such file or directory: 'antiword'
None


In [ ]:
!pip install langchain-qdrant langchain-openai langchain

In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from langchain_text_splitters import CharacterTextSplitter

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings

In [5]:
from langchain_community.vectorstores import FAISS
def process_text(text):
    text = "\n".join([doc.page_content for doc in text])
    # text=""
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=2048,
        chunk_overlap=32,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-small-en-v1.5')
    # embeddings =embeddings
    knowledgeBase = FAISS.from_texts(chunks, embeddings)
    return knowledgeBase

In [6]:
tender_proc_id =67801247
from langchain_community.document_loaders import TextLoader
def load_text_files_from_directory(folder_path):
    all_docs = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            loader = TextLoader(file_path)
            all_docs.extend(loader.load())
    return all_docs

In [7]:
folder_path = f"/data/tendergpt/procurment_guildlines_txt/{tender_proc_id}"
# folder_path = f"/data/QAAPI/DocumentAll (4)/"
print("folder_path:::",folder_path)
all_docs = load_text_files_from_directory(folder_path)
# print("all_docs:::",all_docs)
# knowledge_base = process_text(all_docs)
# print("knowledge_base:::",knowledge_base)

folder_path::: /data/tendergpt/procurment_guildlines_txt/67801247


In [ ]:
import os 
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_qdrant import RetrievalMode
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"
from langchain_community.document_loaders import TextLoader
from langchain_qdrant import QdrantVectorStore
embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-small-en-v1.5')
qdrant = QdrantVectorStore.from_existing_collection(
    embeddings=embeddings,
    collection_name="my_documents",
    retrieval_mode=RetrievalMode.DENSE,
    url="http://localhost:6334",
)

In [2]:
import os
from langchain_qdrant import RetrievalMode
from langchain_community.document_loaders import TextLoader
from langchain_qdrant import QdrantVectorStore
from langchain.embeddings import HuggingFaceEmbeddings

# Set environment variables for CUDA
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-small-en-v1.5')

# Initialize QdrantVectorStore without embeddings
qdrant = QdrantVectorStore.from_existing_collection(
    collection_name="67801247",
    retrieval_mode=RetrievalMode.DENSE,
    url="http://localhost:6334",
)


/home/rajeev/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


ValueError: 'embedding' cannot be None when retrieval mode is 'dense'

In [17]:
query = "who is ramesh sinha"
found_docs = qdrant.similarity_search(query)

In [18]:
found_docs

[Document(page_content='File: Procurement Guidlines_new.pdf\n\nText Data:\nHii\nHello! I am tender tiger bot.How can I assist you today?\nHi\nHi there! How can I help you today? If you have any questions or if there\'s something specific\nyou\'d like to know or discuss, feel free to let me know!\nHey\nHi there! How can I help you today? If you have any questions or if there\'s something specific\nyou\'d like to know or discuss, feel free to let me know!\nHello\nHello! How can I assist you today? If you have any questions or if there\'s something specific\nyou\'d like to discuss, feel free to let me know!\nWho are you?\nI\'m Tendergpt, a virtual assistant created by Tender Tiger. I\'m here to help answer your\nquestions, provide information, or assist you with anything you need to the best of my ability.\nHow can I assist you today?\nAbout TTPL\nTender Tiger Private Limited (TTPL) is the global choice for tender discovery, listing nearly\n100M new business opportunities worldwide. Daily